## PS3

In problem set 1, we illustrated how to perform integration using Monte Carlo Methods. We also showed how to smooth out the function involving sharp peaks by utilizing a predefined weight function to decrease the standard deviation.
In this notebook, we are trying to minimize the variance of the integration further by dividing the integration domain into 10 equal pieces and perform integration for each interval seperately.

To illustrate the idea, we used the following integral

$$\int_{2}^{10}(x-5)\cdot e^{-(\frac{x}{2}-3)}dx$$

For each interval we used linear weight functions ${w_1,w_2,..,w_{10}}$ where $w_i$ is the weight function for i'th interval. We also have the following normalization,

$$\int_{x_{i-1}}^{x_i} w_i(x)dx=1$$ and following boundary conditions,

$$w(x_{i-1})=f(x_{i-1})\text{ }\text{ and }\text{ } w(x_{i})=f(x_{i}) $$


Then the final integral can be calculated as

$$I_{Total}=\sum_{i=1}^{10} I_i$$
where 
$$I_i=\int_{x_{i-1}}^{x_i} f(x)dx$$

and this integral is to be calculated using our standard MC integration method using $w_i(x)$ as the weight function.




Following code snippet illustrates the described method.

In [42]:
import numpy as np
from numpy.polynomial import Polynomial as P
import plotly
import plotly.plotly as py
import plotly.figure_factory as ff

H=0
#Integrand function
def f(x):
    return (x-5)*np.exp(-(x/2-3))+H
    
#Calculates the coefficients of linear weight function.    
def findw(f,lower,upper):
    #Find the linear function.
    slope=(f(upper)-f(lower))/upper-lower
    a=slope
    b=-slope*upper+f(upper)
    #Normalization.
    A=(a/2)*(upper**2)+b*upper-(a/2)*(lower**2)-b*lower
    a/=A
    b/=A
    return [a,b] 

#Performs integration.
def integrate(f,w,lower,upper,N):
    #Generate uniform random inputs.
    inputs=np.random.rand(N)
    
    a=w[0]/2  
    b=w[1]
    c=-(a*lower**2+b*lower)
    
    SUM=0
    SUM2=0
    
    inverse_inputs=[]
    H=30
    for i in inputs:
        p=[(-b-np.sqrt(b**2-4*a*(c-i)))/(2*a),(-b+np.sqrt(b**2-4*a*(c-i)))/(2*a)]
        if p[0]>=lower and p[0]<=upper:
            inverse_inputs.append(p[0])
        else :
            inverse_inputs.append(p[1])

    inverse_inputs=np.array(inverse_inputs)
    #Calculate f(inverse(x))/w(inverse(x)).
    outputsF=f(inverse_inputs)
    outputsW=w[0]*(inverse_inputs)+w[1]
    outputs=outputsF/outputsW
    SUM=outputs.sum()
    SUM2=(outputs*outputs).sum()
    var=SUM2/N-(SUM/N)**2
    var=var/N
    #Store generated points for variance calculation.
    Vsum=outputs.sum()
    return Vsum/N,var
    
#Divide the region into 10 pieces.
l=np.arange(2,10.01,0.8)   
#Real value of the integral
I_real=-16.6728
#N values
N=[10,100,1000,10000,100000,1000000]
#Integration results.
results=[]
#All generated points.
points=[]
#Standart deviation values
sigmas=[]
for k in N:
    I=0
    sigma=0
    for i in range (0,len(l)-1):
        H=50
        w=findw(f,l[i],l[i+1])
        temp,temp2=integrate(f,w,l[i],l[i+1],k)
        I+=temp
        sigma+=temp2
    I=I-H*8
    results.append(I)
    sigmas.append(np.sqrt(sigma))
    print(10*k,I,I-I_real,np.sqrt(sigma),(I-I_real)/np.sqrt(sigma))
  


100 -17.3137004949 -0.640900494879 1.31730988198 -0.486522194698
1000 -16.2364297575 0.436370242492 0.452189910195 0.965015434121
10000 -16.4574275098 0.215372490167 0.143298139305 1.50296780692
100000 -16.6681057488 0.00469425115728 0.0452890031845 0.103651015196
1000000 -16.6797431577 -0.00694315766895 0.0142892987237 -0.485899119559
10000000 -16.6720289166 0.000771083435442 0.00451800941559 0.170668842075


In [43]:
data_matrix = [["N", "I MC", "I Real", "I MC-I Real","sigma","I MC-I Real/sigma"]]
for i in range (0,len(N)):
    element=[10*N[i], results[i], I_real,results[i]-I_real,sigmas[i],(results[i]-I_real)/sigmas[i]]
    data_matrix.append(element)
    
plotly.tools.set_credentials_file(username='guneykan', api_key='Yu3MsgD6Zlfbb0B3S5Mx')
table = ff.create_table(data_matrix)
py.iplot(table)